In [6]:
pip install umap-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from lightfm import LightFM
import umap.umap_ as umap
import matplotlib.pyplot as plt
import scipy.sparse as ss
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances_argmin

In [2]:
from datetime import datetime
from datetime import date
date = date.today()
period = "AUG"
run = f"{date}_{period}"
run

'2023-04-20_AUG'

In [3]:
# %%bigquery users_int_watch
# select event_date, event_timestamp, user_id, device.category as device_type,
#     device.mobile_brand_name as device_brand,
#     device.mobile_model_name as device_model,
#     device.operating_system as device_os,
#     device.operating_system_version as device_os_version,
#     h.key as key,
#     h.value.string_value as client_id,
#     ev.key as event_key,
#     ev.value.string_value as event_params,
#     event_name
# from `vertex-ai-sandbox-380023.analytics_250000498.events_*` 
# left join unnest(user_properties) as h
# left join unnest(event_params) as ev
# where h.key = "client_id"
#     and ev.key = "page_location"
#     and h.value.string_value in (
#         SELECT distinct(h.value.string_value) as client_id,
#         FROM `vertex-ai-sandbox-380023.analytics_250000498.events_*` 
#         left join unnest(user_properties) as h
#         left join unnest(event_params) as ev
#         where h.key = "client_id"
#             and ev.key = "page_location"
#             and ev.value.string_value like '%watch%'
#             and h.value.string_value is not null
#             and event_date between format_date("%Y%m%d", date_sub(date_sub(current_date(), interval 49 day), interval 3 month))
#             and format_date("%Y%m%d", date_sub(current_date(), interval 49 day)))
#     and event_date between format_date("%Y%m%d", date_sub(date_sub(current_date(), interval 49 day), interval 3 month))
#     and format_date("%Y%m%d", date_sub(current_date(), interval 49 day))

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%%bigquery users_int_watch
select event_date, event_timestamp, user_id, device.category as device_type,
    device.mobile_brand_name as device_brand,
    device.mobile_model_name as device_model,
    device.operating_system as device_os,
    device.operating_system_version as device_os_version,
    h.key as key,
    h.value.string_value as client_id,
    ev.key as event_key,
    ev.value.string_value as event_params,
    event_name
from `vertex-ai-sandbox-380023.analytics_250000498.events_*` 
left join unnest(user_properties) as h
left join unnest(event_params) as ev
where h.key = "client_id"
    and ev.key = "page_location"
    and h.value.string_value in (
        SELECT distinct(h.value.string_value) as client_id,
        FROM `vertex-ai-sandbox-380023.analytics_250000498.events_*` 
        left join unnest(user_properties) as h
        left join unnest(event_params) as ev
        where h.key = "client_id"
            and ev.key = "page_location"
            and ev.value.string_value like '%watch%'
            and h.value.string_value is not null
            and event_date between "20220810"
            and "20220930")
     and event_date between "20220810" and "20220930"

Query is running:   0%|          |

Downloading:   0%|          |

In [1]:
%%bigquery users_int_watch


select event_date, event_timestamp, user_id, device_type,
    device_brand,
    device_os,
    client_id, event_params
from `vertex-ai-sandbox-380023.feature_prep.watch_watchers`

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
users_int_watch.to_parquet("users_watches_AUG.pq")

In [10]:
users = users_int_watch.groupby(["client_id"]).user_id.count().reset_index()
users = users.rename(columns={"user_id": "is_user"})
users.loc[users["is_user"] > 0, "is_user"] = 1

In [ ]:
# non_use_mask = users_int_watch["user_id"].isna()
# non_users = users_int_watch[non_use_mask].copy()
# non_users["is_user"] = 0
# users = users_int_watch[~non_use_mask].copy()
# users["is_user"] = 1
# size_df = users_int_watch.shape[0]
# print("users_int_watch was: ", users_int_watch.shape[0], "events")
# del users_int_watch
# users_int_watch = pd.concat([non_users, users])
# del non_users, users
# if size_df == users_int_watch.shape[0]:
#     print("new prep_df size is correct and equals", size_df)
# else:
#     print("there is an error, new size is", size_df-users_int_watch.shape[0], "entries smaller")

In [ ]:
# users_or_non = users_int_watch[["client_id", "is_user"]].drop_duplicates()
# df_of_dup = users_or_non.groupby(["client_id"]).is_user.count().reset_index().sort_values(by="is_user", ascending=False)
# list_of_dup = list(df_of_dup[df_of_dup["is_user"] == 2]["client_id"])
# users_or_non = users_or_non.drop_duplicates(subset="client_id")
# users_or_non.loc[users_or_non["client_id"].isin(list_of_dup), "is_user"] = 1

# del df_of_dup, list_of_dup

In [11]:
user_hits = users_int_watch[["event_timestamp", "client_id"]]\
    .groupby(["client_id"]).event_timestamp.count()\
    .reset_index()
users_watches = users_int_watch[users_int_watch["event_params"].str.contains("watch")]
users_watches_hits = users_watches[["event_timestamp", "client_id"]]\
    .groupby(["client_id"]).event_timestamp.count()\
    .reset_index()
user_hits = user_hits.rename(columns={"event_timestamp": "hits"})
users_watches_hits = users_watches_hits.rename(columns={"event_timestamp": "hits_watches"})
user_hits_full = user_hits.merge(users_watches_hits, on="client_id", how="inner")

users_active = users_int_watch[["client_id", "event_date"]].groupby(["client_id"])\
    .agg({"event_date": pd.Series.nunique})\
    .reset_index()

In [12]:
users_active = users_active.rename(columns={"event_date": "days_active"})
users_num = users.merge(user_hits_full, on="client_id", how="inner")\
    .merge(users_active, on="client_id", how="inner")

In [13]:
run

'2023-04-20_AUG'

In [14]:
users_num.to_parquet(f"art/users_numerical_activity_{run}.pq")

In [15]:
users_device_brand_watch = users_int_watch[["client_id", "device_brand", "event_timestamp"]].copy()
users_device_brand_watch["device_brand"] = users_device_brand_watch["device_brand"].replace("", "brand_unknown")

# collecting device brand to features
users_int_watch_devices = users_int_watch[["client_id", "device_brand", "event_timestamp"]].drop_duplicates()
pivot_brand = pd.pivot_table(users_int_watch_devices, index=["client_id"],
                             columns=["device_brand"], values="event_timestamp",
                             aggfunc="count", fill_value=0).reset_index()
del users_device_brand_watch


In [16]:
# collecting device type
users_int_watch_devices = users_int_watch[["client_id", "device_type", "event_timestamp"]].drop_duplicates()
pivot_devices = pd.pivot_table(users_int_watch_devices, index=["client_id"],
                               columns=["device_type"], values="event_timestamp",
                               aggfunc="count", fill_value=0).reset_index()

del users_int_watch_devices

In [17]:
pivot_devices.to_parquet(f"art/activity/cid_device_type_{run}.pq")
pivot_brand.to_parquet(f"art/activity/cid_device_brand_{run}.pq")

In [18]:
# collecting device related features together

user_device_features = pivot_devices\
    .merge(pivot_brand, on="client_id", how="outer")\
    .reset_index(drop=True)\
    .fillna(0)

for col in user_device_features.drop(columns=["client_id"]).columns:
    user_device_features[col] = user_device_features[col].astype("int")

del users_int_watch

In [19]:
user_device_features.to_parquet(f"users_device_features_{run}.pq")

In [3]:
user_device_features = pd.read_parquet(f"users_device_features_{run}.pq")

In [4]:
sparse_device = ss.csr_matrix(user_device_features.drop(columns=["client_id"]).values)

In [5]:
if "model" in locals():
    del model
n_comp = 5
# defining the LightFM model
model = LightFM(loss='warp', random_state=21, no_components=n_comp)
# fitting
model.fit(sparse_device, epochs=20, num_threads=1)
# saving embeddings
user_emb_dev = model.user_embeddings

In [6]:
emb_dev = pd.DataFrame(user_emb_dev)
for col in emb_dev.columns:
    emb_dev = emb_dev.rename(columns={col: f"col_{col}"})

In [7]:

emb_dev.to_parquet(f"emb_devices_{n_comp}.pq")

In [8]:
user_emb_dev = pd.read_parquet("dev_emb.pq").values
user_emb_dev

array([[-0.15831123,  0.16300143,  0.08756991, -0.08848318, -0.0199209 ],
       [-0.25290528, -0.50800145,  0.39204526, -0.37828234, -0.03308587],
       [ 0.02619106,  0.13684411,  0.22933   , -0.06309661, -0.04911786],
       ...,
       [-0.10125699,  0.10553132,  0.06046237, -0.02471155,  0.01109862],
       [-0.1900819 , -0.00957474,  0.04734597, -0.09362752,  0.14333741],
       [-0.45053533,  0.60057545,  0.13223307, -0.10930066,  0.00075004]],
      dtype=float32)

In [7]:
umap_model = umap.UMAP(n_components=2, metric='euclidean', n_neighbors=30)

In [ ]:
embedding = umap_model.fit_transform(user_emb_dev)

In [ ]:
if "umap_model" in locals():
    del umap_model

# defining the UMAP model
umap_model = umap.UMAP(n_components=2, random_state=21, metric='euclidean', n_neighbors=30)

# fitting
embedding = umap_model.fit_transform(user_emb_dev)

# plotting
plt.scatter(embedding[:, 0], embedding[:, 1])
plt.show()